In [34]:
import pandas as pd
import numpy as np
import re #Regex lib
from forex_python.converter import CurrencyRates #currencies lib
from datetime import datetime
import matplotlib

### Read in the dataframes and create a dict {(year, dataframe)}


In [35]:
# Create a list of filenames for the datasets
years = [2017, 2018, 2019, 2020, 2021]
file_names = ['datasets/{}.csv'.format(year) for year in years]

# Read in the dataframes and create a dict {(year, dataframe)}
data_frames_dict = {}

data_frames_dict[2017] = pd.read_csv('datasets/2017.csv', encoding = "ISO-8859-1").filter(items=['Country', 'FormalEducation']).dropna(thresh=2) 
data_frames_dict[2018] = pd.read_csv('datasets/2018.csv', encoding = "ISO-8859-1").filter(items=['Country',  'FormalEducation']).dropna(thresh=2) 
data_frames_dict[2019] = pd.read_csv('datasets/2019.csv', encoding = "ISO-8859-1").filter (items=['Country',  'EdLevel']).dropna(thresh=2) 
data_frames_dict[2020] = pd.read_csv('datasets/2020.csv', encoding = "ISO-8859-1").filter (items=['Country',  'EdLevel']).dropna(thresh=2) 
data_frames_dict[2021] = pd.read_csv('datasets/2021.csv', encoding = "ISO-8859-1").filter (items=['Country',  'EdLevel']).dropna(thresh=2) 


/var/folders/h2/9wb_tlwj1bd2smbvq_25k5z4097jdb/T/ipykernel_15653/1446315357.py:9: DtypeWarning: Columns (8,12,13,14,15,16,50,51,52,53,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128) have mixed types. Specify dtype option on import or set low_memory=False.
  data_frames_dict[2018] = pd.read_csv('datasets/2018.csv', encoding = "ISO-8859-1").filter(items=['Country',  'FormalEducation']).dropna(thresh=2)


In [36]:
ed_dict =  {'Secondary school': 'Secondary school',
 "Some college/university study without earning a bachelor's degree" : "college/university without bachelor's degree",
 "Bachelor's degree": "Bachelor's degree",
 'Doctoral degree': 'Doctoral degree',
 "Master's degree": "Master's degree",
 'Professional degree': "Professional degree",
 'Primary/elementary school': 'Primary/elementary school',
 'I prefer not to answer': 'I prefer not to answer',
 'I never completed any formal education': 'No formal education', 
 'Bachelorâ\x80\x99s degree (BA, BS, B.Eng., etc.)': "Bachelor's degree",
 'Associate degree': 'Associate degree',
 'Some college/university study without earning a degree': "college/university without bachelor's degree",
 'Masterâ\x80\x99s degree (MA, MS, M.Eng., MBA, etc.)': "Master's degree",
 'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 'Secondary school',
 'Professional degree (JD, MD, etc.)': "Professional degree",
 'I never completed any formal education': 'No formal education', 
 'Other doctoral degree (Ph.D, Ed.D., etc.)':  'Doctoral degree',
 'Associate degree (A.A., A.S., etc.)': 'Associate degree',
 'Something else': 'Something else', 
 'Masterâ\x80\x99s degree (M.A., M.S., M.Eng., MBA, etc.)': "Master's degree",
 'Bachelorâ\x80\x99s degree (B.A., B.S., B.Eng., etc.)' : "Bachelor's degree",
 'Other doctoral degree (Ph.D., Ed.D., etc.)' : 'Doctoral degree'
}


    


### obtain developer types 


In [37]:
import codecs
import json
for year, df in data_frames_dict.items():
    df.columns = ['Country',  'EdLevel']
  
    

    
    # get unique satisfactions
    #print(df.JobSatisfaction.unique())
   
    df['EdLevel'] = df.EdLevel.apply(lambda x: ed_dict[x])
        
    df2 = df.copy(deep=True)
       
    df['EdLevelCount'] = pd.Series(np.ones(len(df.EdLevel),dtype= np.int32), index=df.index)  
    df = df.groupby(by=['Country','EdLevel'] , as_index=False).sum()
   
    
    
    json_dict = {}
    for k, v in df.groupby('Country'):
        inner_dict = {}
        for sat, count in zip(v['EdLevel'], v['EdLevelCount']):
            inner_dict[sat] = count 
        json_dict[k] = inner_dict
        
 
        
    with codecs.open("processed_data/education/{}-education_count_per_country.json".format(year), "w", encoding='utf-8') as outfile:
        json.dump(json_dict, outfile, indent = 4, ensure_ascii=False)

    
    df2.columns = ['Country', 'respondentsCount']

    df2 = df2.groupby(by=['Country'] , as_index=False).count()
    
    print(df2.shape)
        
    print(df2.head())
    
    json_dict2 = {}
    for index, row in df2.iterrows():
        json_dict2[row["Country"]] = row["respondentsCount"]
  

    
  
    with codecs.open("processed_data/education/{}-education_respondents_count_per_country.json".format(year), "w", encoding='utf-8') as outfile:
        json.dump(json_dict2, outfile, indent = 4, ensure_ascii=False)



(201, 2)
          Country  respondentsCount
0     Afghanistan                60
1   Aland Islands                22
2         Albania                76
3         Algeria                41
4  American Samoa                 5
(183, 2)
       Country  respondentsCount
0  Afghanistan                53
1      Albania                99
2      Algeria               116
3      Andorra                12
4       Angola                 8
(178, 2)
       Country  respondentsCount
0  Afghanistan                39
1      Albania                81
2      Algeria               122
3      Andorra                 7
4       Angola                 3
(179, 2)
       Country  respondentsCount
0  Afghanistan                60
1      Albania                46
2      Algeria                77
3      Andorra                 6
4       Angola                 5
(181, 2)
       Country  respondentsCount
0  Afghanistan                61
1      Albania                71
2      Algeria                46
3      Andorr